<a href="https://colab.research.google.com/github/ColeBromfield01/bromfield-portfolio/blob/main/DATA604_Homework_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first step is to import the USPS dataset.  This can be done through scikit-learn.

In [ ]:
from sklearn.datasets import fetch_openml

# Load USPS dataset
usps = fetch_openml('usps', version=2)

# Access the data and labels
X = usps.data
y = usps.target

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Now that the dataset has been imported, we can set up our kernel PCA transformation.  With the high dimensionality of the dataset, I determined RBF to be the most appropriate kernel method.  RBF is less reliant on underlying patterns in the data, which would have been difficult to determine in this case.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    usps.data, usps.target, test_size=0.2, random_state=42
)

kpca = KernelPCA(kernel='rbf', gamma=0.03)
kpca.fit(X_train)

X_train_kpca = kpca.transform(X_train)
X_test_kpca = kpca.transform(X_test)

With the data transformed, we can now perform kNN with k = 20.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

k = 20
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_kpca, y_train)

y_pred = knn.predict(X_test_kpca)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9483870967741935


We see a very high accuracy with this method, at nearly 95%.  To determine how significantly we can lower the dimensionality, we can use an iterative method such as bisection.  With this method, we can calculate the smallest number of principal components that can be kept without the accuracy dropping below a certain threshold.  Since we don't want to lose too much accuracy, let's set that threshold at 94%.

In [ ]:
start = 1
end = X_train_kpca.shape[1]
min_dimensionality = None
accuracy_threshold = 0.94

# Perform bisection method
while start <= end:
    mid = (start + end) // 2

    X_train_kpca_reduced = X_train_kpca[:, :mid]
    X_test_kpca_reduced = X_test_kpca[:, :mid]

    knn.fit(X_train_kpca_reduced, y_train)
    y_pred = knn.predict(X_test_kpca_reduced)
    accuracy = accuracy_score(y_test, y_pred)

    if accuracy >= accuracy_threshold:
        min_dimensionality = mid
        end = mid - 1
    else:
        start = mid + 1

# Print the final dimensionality and accuracy
print("Final Dimensionality:", min_dimensionality)
print("Final Accuracy:", accuracy)

Final Dimensionality: 45
Final Accuracy: 0.9424731182795699


Surprisingly, the data can be reduced to 45 dimensions with less than a 1% drop in accuracy.  This is a significant drop from the 256 dimensions of the raw data (and even more so compared with the thousands of dimensions in the RBF-transformed data).